In [1]:
import nltk
from nltk.tokenize import word_tokenize 
import pandas as pd
import os
import json
import glob
import spacy
nlp = spacy.load('pt')



jsons=[]
for filename in glob.iglob("D:\\socialbus\\users\\*" + '*', recursive=True):
    try:
        jsons.append(pd.read_json(filename,lines=True))
       # print(filename)
        #print("deu")
    except:
        pass
        
print(str(len(jsons)) + " json files found")  

36 json files found


In [34]:
i=0
flagged=[]
teste=[]
for json in jsons:
    i=len(json)-1
    while(i>1):
        
        
        
        print(json['text'][i])
        print("Time of Comment: " + str(get_time_of_comment(json['created_at'][i])))
        print("Number of Stopwords: "+ str(get_stopword_nr(json['text'][i])))
        print("Number of Hashtags: "+ str(get_nr_of_hashtags(json['metadata'][i])))
        print("Number of Mentions: "+ str(get_nr_of_mentions(json['metadata'][i])))
        print("Number of Urls: "+ str(get_nr_of_urls(json['metadata'][i])))
        print("Number of Emoticons: "+ str(get_nr_of_emoticons(json['metadata'][i])))
        print("Number of Swears: "+ str(get_swear_count(json['text'][i])))
        print("Number of Favorites: "+ str(get_favorites_count(json['favorite_count'][i])))
        print("Number of Verbs: "+ str(get_verb_count(json['text'][i])))
        print("Number of Nouns: "+ str(get_noun_count(json['text'][i])))
        print("Number of Adjectives: "+ str(get_adj_count(json['text'][i])))
              
        print("\n -------------------------------------------------------\n")
        
        
        
        
        i=i-1
   




#get_nr_of_hashtags(jsonLine)        
#get_nr_of_mentions(jsonLine) 
#get_nr_of_urls(jsonLine) 
#get_nr_of_emoticons(jsonLine) 
#get_swear_count(jsonLine) 
#get_favorites_count(jsonLine) 
#get_verb_count(jsonLine) 
#get_noun_count(jsonLine) 
#get_adj_count(jsonLine) 

@Luis_Eusebio @1963Cor Mas para ti só as iniciativas legislativas do PS é que contam na AR? Oh pá, há vários partid… https://t.co/mR89ra4Lru
Time of Comment: 17:59:54
Number of Stopwords: 8
Number of Hashtags: 0
Number of Mentions: 2
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 3
Number of Adjectives: 2

 -------------------------------------------------------

o problema de fazer sexo com luz apagada, é que nunca sabes se ela está excitada ou com o período https://t.co/uAflSLCYSm
Time of Comment: 17:59:51
Number of Stopwords: 11
Number of Hashtags: 0
Number of Mentions: 0
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 6
Number of Nouns: 5
Number of Adjectives: 0

 -------------------------------------------------------

@SICNoticias #neto #neta #tio #tia #bola #futebol #portuguesa #portugues #familia #fe #amor #vida #esperanca… https://t.co/1SdNTGuLdW
Time o

Number of Stopwords: 7
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 3
Number of Adjectives: 0

 -------------------------------------------------------

RT @captomente: O plano de Varandas &amp; C.ia até ao momento:

- vender Raphinha
- vender Bas Dost
- vender Bruno Fernandes
- vender/emprestar…
Time of Comment: 17:59:16
Number of Stopwords: 3
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 2
Number of Adjectives: 0

 -------------------------------------------------------

RT @AAAAAGGHHHH: https://t.co/htIdhAyLlq
Time of Comment: 17:59:12
Number of Stopwords: 0
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 0
Number of Adjectives: 0

 ---

Number of Adjectives: 0

 -------------------------------------------------------

@PNSpedronuno @JoedwartJoe E porque será que os empresários estão sempre ricos e a empresa e os trabalhadores não g… https://t.co/FnAkfhZPuo
Time of Comment: 17:58:49
Number of Stopwords: 9
Number of Hashtags: 0
Number of Mentions: 2
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 1
Number of Adjectives: 1

 -------------------------------------------------------

@lilianappinto Está há demasiado tempo parada, bem mais de 3 anos, mas a culpa é minha. Acomodo-me e aceito isso... beijinho, Lil 😘
Time of Comment: 17:58:49
Number of Stopwords: 9
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 5
Number of Adjectives: 0

 -------------------------------------------------------

Here’s a song for you… Houdini by Foster The Peo

Number of Adjectives: 0

 -------------------------------------------------------

#FutsalSCP | 🎙 33' Que defesa de Gonçalo Portugal 👌

#ADFSCP | 1-2 | #CadaJogoUmaFinal
Time of Comment: 17:57:35
Number of Stopwords: 1
Number of Hashtags: 3
Number of Mentions: 0
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 1
Number of Adjectives: 0

 -------------------------------------------------------

RT @TheRealJuliaMNG: Se mais provas fossem precisas para demonstrar como a porra de um clube controla e manda nisto tudo, nomeadamente nos…
Time of Comment: 17:57:34
Number of Stopwords: 8
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 3
Number of Nouns: 3
Number of Adjectives: 2

 -------------------------------------------------------

@Chartersfil @joaomiranda @cryptolobopt @refe56 @pedroml No caso do aborto fizemos um referend

Number of Adjectives: 1

 -------------------------------------------------------

RT @onossopalestra: "O Divino poderia ser meu pai. Mas ele mais parecia meu bisneto nessa tarde em casa. Mais que parabéns, uma vez mais, m…
Time of Comment: 17:56:52
Number of Stopwords: 8
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 8
Number of Adjectives: 0

 -------------------------------------------------------

RT @JoseAdelinoAlm2: Este movimento tem q crescer.! https://t.co/Z06TBFpv1F
Time of Comment: 17:56:49
Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 2
Number of Adjectives: 0

 -------------------------------------------------------

@tomasrdacunha Epa, sinto que este é daqueles casos em que cheguei antes de toda a gente mas que lhe fui

...

Number of Stopwords: 0
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 1
Number of Adjectives: 0

 -------------------------------------------------------

RT @wandson: vou bem ali em Aveiro e já volto. https://t.co/U9uLgDb7u0
Time of Comment: 18:09:27
Number of Stopwords: 3
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 1
Number of Adjectives: 0

 -------------------------------------------------------

RT @SPORTTVPortugal: Míssil teleguiado de @B_Fernandes8  só podia resultar em golo. Voltaremos a ver algo do género quando o @ManUtd  visit…
Time of Comment: 18:09:16
Number of Stopwords: 7
Number of Hashtags: 0
Number of Mentions: 3
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 3
Num

Number of Adjectives: 0

 -------------------------------------------------------

RT @maldsuado: País de corruptos foda-se... https://t.co/LaOjf9nHCD
Time of Comment: 18:08:48
Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 1
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 2
Number of Adjectives: 0

 -------------------------------------------------------

RT @JoseCasteloBra2: Orelha grande é orelha de Rica! Por isso não se queixe! É sinal que é rico !
Time of Comment: 18:08:46
Number of Stopwords: 7
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 1
Number of Adjectives: 2

 -------------------------------------------------------

Covid-19: Grupo de repatriados da China termina quarentena e já deixou o Hospital Pulido Valente… https://t.co/0rhRdApooT
Time of Comment: 18:08:44
Number 

Number of Verbs: 1
Number of Nouns: 0
Number of Adjectives: 0

 -------------------------------------------------------

@K2_Tudor_Run atira te ks ks ks qéro ver 👀👀
Time of Comment: 18:08:13
Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 1
Number of Adjectives: 0

 -------------------------------------------------------

@ppmascarenhas Todos os burros comem a palha que lhe dão. O VAR confirma ou anula com base em linhas e em imagens v… https://t.co/djjmdZgLfp
Time of Comment: 18:08:10
Number of Stopwords: 9
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 3
Number of Nouns: 6
Number of Adjectives: 0

 -------------------------------------------------------

RT @TheRealJuliaMNG: Agora... é lidar!

Rui Pinto tentou dar provas à Justiça, mas foi ignorado, a

Number of Adjectives: 0

 -------------------------------------------------------

@tvi24pt Passamos a pagar por sacos plasticos.
Time of Comment: 18:07:42
Number of Stopwords: 2
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 1
Number of Adjectives: 1

 -------------------------------------------------------

Alisson Clemence... que GUARDIÃO https://t.co/yHchSYo2Vw
Time of Comment: 18:07:41
Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 0
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 0
Number of Adjectives: 0

 -------------------------------------------------------

@rpOliveira @CapitaoBina A mim tambem me bloqueou. Depois de me chamar de snowflake por ter, como tu, discordado do tweet dele
Time of Comment: 18:07:40
Number of Stopwords: 9
Number of Hashtags: 0
Number of Mentions:

KeyboardInterrupt: 

In [33]:
def get_stopword_nr(sentence):
    
    word_tokens = word_tokenize(sentence)
    count=0
    
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for stopword in stopwords:
        for word in word_tokens:
            if stopword==word:
                count+=1
       
    
    return count
    
    

In [7]:
def get_time_of_comment(jsonLine):
     
    dateStr=str(jsonLine)
       
    fullDateArray=dateStr.split()
    time=fullDateArray[1].split('+')
    
    return time[0]
    

In [8]:
def get_nr_of_hashtags(jsonLine):
    return len(jsonLine['hashtags'])

In [9]:
def get_nr_of_mentions(jsonLine):
    return len(jsonLine['mentions'])

In [10]:
def get_nr_of_urls(jsonLine):
    return len(jsonLine['urls'])

In [11]:
def get_nr_of_emoticons(jsonLine):
    return len(jsonLine['emoticons'])

In [32]:
def get_swear_count(jsonLine):
    swearCount=0
    swears=['broch','cabrões','cabr','cagalhão','caralh','cona','corn','enrabar','cuzinho','esporra','fod','mamadas','panasca','peid','piça','picha','pila','tomates','puta','merd','caralho','crl','fds','fdç','paneleir']
    for swear in swears:
        if swear.lower() in jsonLine.lower():
            swearCount+=1
            
    return swearCount        

In [24]:
def get_favorites_count(jsonLine):
    return jsonLine

In [16]:
def get_day_of_tweet(jsonLine):
    dateStr=str(jsonLine)
       
    fullDateArray=dateStr.split()

In [17]:
def get_verb_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    verbCount=0


    for token in doc:
        if token.pos_=='VERB':
            verbCount+=1
        
    return verbCount

In [26]:
def get_noun_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    nounCount=0


    for token in doc:
        if token.pos_=='NOUN' or token.pos=='PROPN':
            nounCount+=1
        
    return nounCount

In [27]:
def get_adj_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    adjCount=0


    for token in doc:
        if token.pos_=='ADJ':
            adjCount+=1
        
    return adjCount

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(s)? (<ipython-input-31-dd4395da4df5>, line 18)